# Libraries

In [1]:
import sqlite3
import numpy as np
from collections import OrderedDict, namedtuple
from distutils.spawn import find_executable
import matplotlib.pyplot as plt

import sys
import subprocess
import pathlib
import tempfile
import importlib
import os
import time
import random
import pickle

import math

from configparser import ConfigParser

# from google.protobuf.json_format import MessageToDict

In [2]:
# DEVICE = 0

import os
import time
import pickle
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd

import torch
torch.backends.cudnn.benchmark = True
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils import data

# torch.cuda.set_device(DEVICE)
torch.cuda.empty_cache() 
torch.backends.cudnn.benchmark=True

In [3]:
np.random.seed(0)
torch.manual_seed(0)


import os
os.environ['CUDA_VISIBLE_DEVICES']='1'

# load the dataset

In [4]:
with open('scrimmage4_link_dataset.pickle', 'rb') as file:
    link_dataset = pickle.load(file)

In [5]:
train_data = link_dataset[:200] 
test_data = link_dataset[200:]

In [6]:
train_x = torch.cat(tuple(link[0] for link in train_data),dim=0)
train_y = torch.cat(tuple(link[1] for link in train_data),dim=0)

test_x = torch.cat(tuple(link[0] for link in test_data),dim=0)
test_y = torch.cat(tuple(link[1] for link in test_data),dim=0)

In [7]:
test_x.size()

torch.Size([2046930, 20])

In [8]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()

        self.linear1 = nn.Linear(20, 128)
        self.linear2 = nn.Linear(128, 128)
        self.linear3 = nn.Linear(128, 128)
        self.linear4 = nn.Linear(128, 128)
        self.linear5 = nn.Linear(128, 2)
        
        self.relu = nn.ReLU()
        
#         self.dropout = nn.Dropout(0.6)

    
    def forward(self, input):

        output = self.linear1(input)
        output = self.relu(output)
#         output = self.dropout(output)

        output = self.linear2(output)
        output = self.relu(output)
#         output = self.dropout(output)

        output = self.linear3(output)
        output = self.relu(output)
#         output = self.dropout(output)

        output = self.linear4(output)
        output = self.relu(output)
#         output = self.dropout(output)

        output = self.linear5(output)
        
        return output

# Train Model

In [9]:
NUM_EPOCHS = 10
BATCH_SIZE = 1024

model = MLP()
model.cuda()

loss_function = nn.CrossEntropyLoss(weight=torch.tensor([1.0,1.0])).cuda()
optimizer = optim.Adam(model.parameters(), 0.001)
# optimizer = optim.SGD(model.parameters(), lr=0.001)

train_dataloader = data.DataLoader(
    dataset=data.TensorDataset(train_x,train_y), 
    batch_size=BATCH_SIZE, shuffle=True, num_workers=16, pin_memory=True)

test_dataloader = data.DataLoader(
    data.TensorDataset(test_x,test_y), 
    batch_size=BATCH_SIZE, shuffle=False, num_workers=16, pin_memory=True)

model.train()
for epoch_idx in range(NUM_EPOCHS):  # again, normally you would NOT do 300 epochs, it is toy data

    progress_training_epoch = tqdm(
        train_dataloader, 
        desc=f'Epoch {epoch_idx}/{NUM_EPOCHS}, Training',
        miniters=1, ncols=88, position=0,
        leave=True, total=len(train_dataloader), smoothing=.9)

    progress_validation_epoch = tqdm(
        test_dataloader, 
        desc=f'Epoch {epoch_idx}/{NUM_EPOCHS}, Validation',
        miniters=1, ncols=88, position=0, 
        leave=True, total=len(test_dataloader), smoothing=.9)

    train_loss = 0
    train_size = 0
    for idx, (input, target) in enumerate(progress_training_epoch):
        input = input.cuda()
        target = target.cuda()
        model.zero_grad()
        predict = model(input)
        loss = loss_function(predict, target)
        loss.backward()
        optimizer.step()
        train_loss += loss * target.size()[0]
        train_size += target.size()[0]

    test_loss = 0
    test_size = 0
    predict = []
    target = []
    model.eval()
    with torch.no_grad():
        for idx, (batch_input, batch_target) in enumerate(progress_validation_epoch):
            batch_input = batch_input.cuda()
            batch_target = batch_target.cuda()
            batch_predict = model(batch_input)
            loss = loss_function(batch_predict, batch_target)
            predict.append(batch_predict.argmax(dim=1).cpu().numpy())
            target.append(batch_target.cpu().numpy())        
            test_loss += loss * batch_target.size()[0]
            test_size += batch_target.size()[0]
    predict = np.concatenate(predict, axis=0)
    target = np.concatenate(target, axis=0)
    

#     print(f'train loss:{train_loss.item()/train_size: .5f}, '
#           f'test loss:{test_loss.item()/test_size: .5f}, '
#           f'overall accuracy:{(predict == target).sum() / target.size}, '
#           f'positive accuracy:{predict[target==1].sum() / target.sum()}, '
#           f'negative accuracy:{(np.logical_not(target).sum() - predict[np.logical_not(target)].sum()) / np.logical_not(target).sum()}')

    print(f'train loss:{train_loss.item()/train_size: .5f}, '
          f'test loss:{test_loss.item()/test_size: .5f}')

Epoch 1/10, Validation:   0%|                                  | 0/1999 [00:00<?, ?it/s]

train loss: 0.44490, test loss: 0.64540


Epoch 2/10, Validation:   0%|                                  | 0/1999 [00:00<?, ?it/s]

train loss: 0.41472, test loss: 0.65335


Epoch 3/10, Validation:   0%|                                  | 0/1999 [00:00<?, ?it/s]

...

train loss: 0.39950, test loss: 0.67871


Epoch 9/10, Validation:   0%|                                  | 0/1999 [00:00<?, ?it/s]

train loss: 0.39884, test loss: 0.67976


Epoch 9/10, Validation: 100%|███████████████████████| 1999/1999 [00:46<00:00, 43.43it/s]


train loss: 0.39819, test loss: 0.67808


In [10]:
test_dataloader = data.DataLoader(
    data.TensorDataset(test_x,test_y), 
    batch_size=BATCH_SIZE, shuffle=False, num_workers=16, pin_memory=True)

predict = []
target = []
model.eval()
with torch.no_grad():
    for idx, (batch_input, batch_target) in enumerate(progress_validation_epoch):
        batch_input = batch_input.cuda()
        batch_target = batch_target.cuda()
        batch_predict = model(batch_input)
        predict.append(batch_predict.argmax(dim=1).cpu().numpy())
        target.append(batch_target.cpu().numpy())        
predict = np.concatenate(predict, axis=0)
target = np.concatenate(target, axis=0)

tp = predict[target==1].sum()
tn = (target==0).sum() - predict[target==0].sum()
fp = predict[target==0].sum()
fn = (target==1).sum() - predict[target==1].sum()

print(f'True Positive:{tp}, '
      f'True Negative:{tn}, '
      f'False Positive:{fp}, '
      f'False Negative:{fn}')

True Positive:1088615, True Negative:251322, False Positive:440096, False Negative:266897


In [11]:
accr = (tp+tn)/(tp+tn+fp+fn)
posaccr = tp/(tp+fp)
negaccr = tn/(tn+fn)
print(f'Accuracy:{accr}, '
      f'Positive Accuracy:{posaccr}, '
      f'Negative Accuracy:{negaccr}, '
      f'Weighted Accuracy:{0.5*(posaccr+negaccr)}')

Accuracy:0.6546081204535573, Positive Accuracy:0.7121130154751291, Negative Accuracy:0.4849725695121175, Weighted Accuracy:0.5985427924936233


# Train Model 1 Epoch

In [12]:
NUM_EPOCHS = 1
BATCH_SIZE = 1024

model = MLP()
model.cuda()

loss_function = nn.CrossEntropyLoss(weight=torch.tensor([1.0,1.0])).cuda()
optimizer = optim.Adam(model.parameters(), 0.001)
# optimizer = optim.SGD(model.parameters(), lr=0.001)

train_dataloader = data.DataLoader(
    dataset=data.TensorDataset(train_x,train_y), 
    batch_size=BATCH_SIZE, shuffle=True, num_workers=16, pin_memory=True)

test_dataloader = data.DataLoader(
    data.TensorDataset(test_x,test_y), 
    batch_size=BATCH_SIZE, shuffle=False, num_workers=16, pin_memory=True)

model.train()
for epoch_idx in range(NUM_EPOCHS):  # again, normally you would NOT do 300 epochs, it is toy data

    progress_training_epoch = tqdm(
        train_dataloader, 
        desc=f'Epoch {epoch_idx}/{NUM_EPOCHS}, Training',
        miniters=1, ncols=88, position=0,
        leave=True, total=len(train_dataloader), smoothing=.9)

    progress_validation_epoch = tqdm(
        test_dataloader, 
        desc=f'Epoch {epoch_idx}/{NUM_EPOCHS}, Validation',
        miniters=1, ncols=88, position=0, 
        leave=True, total=len(test_dataloader), smoothing=.9)

    train_loss = 0
    train_size = 0
    for idx, (input, target) in enumerate(progress_training_epoch):
        input = input.cuda()
        target = target.cuda()
        model.zero_grad()
        predict = model(input)
        loss = loss_function(predict, target)
        loss.backward()
        optimizer.step()
        train_loss += loss * target.size()[0]
        train_size += target.size()[0]

    test_loss = 0
    test_size = 0
    predict = []
    target = []
    model.eval()
    with torch.no_grad():
        for idx, (batch_input, batch_target) in enumerate(progress_validation_epoch):
            batch_input = batch_input.cuda()
            batch_target = batch_target.cuda()
            batch_predict = model(batch_input)
            loss = loss_function(batch_predict, batch_target)
            predict.append(batch_predict.argmax(dim=1).cpu().numpy())
            target.append(batch_target.cpu().numpy())        
            test_loss += loss * batch_target.size()[0]
            test_size += batch_target.size()[0]
    predict = np.concatenate(predict, axis=0)
    target = np.concatenate(target, axis=0)

#     print(f'train loss:{train_loss.item()/train_size: .5f}, '
#           f'test loss:{test_loss.item()/test_size: .5f}, '
#           f'overall accuracy:{(predict == target).sum() / target.size}, '
#           f'positive accuracy:{predict[target==1].sum() / target.sum()}, '
#           f'negative accuracy:{(np.logical_not(target).sum() - predict[np.logical_not(target)].sum()) / np.logical_not(target).sum()}')

    print(f'train loss:{train_loss.item()/train_size: .5f}, '
          f'test loss:{test_loss.item()/test_size: .5f}')

Epoch 0/1, Validation: 100%|████████████████████████| 1999/1999 [00:34<00:00, 57.30it/s]

train loss: 0.44307, test loss: 0.65878


# Test

In [13]:
test_dataloader = data.DataLoader(
    data.TensorDataset(test_x,test_y), 
    batch_size=BATCH_SIZE, shuffle=False, num_workers=16, pin_memory=True)

predict = []
target = []
model.eval()
with torch.no_grad():
    for idx, (batch_input, batch_target) in enumerate(progress_validation_epoch):
        batch_input = batch_input.cuda()
        batch_target = batch_target.cuda()
        batch_predict = model(batch_input)
        predict.append(batch_predict.argmax(dim=1).cpu().numpy())
        target.append(batch_target.cpu().numpy())        
predict = np.concatenate(predict, axis=0)
target = np.concatenate(target, axis=0)

tp = predict[target==1].sum()
tn = (target==0).sum() - predict[target==0].sum()
fp = predict[target==0].sum()
fn = (target==1).sum() - predict[target==1].sum()

print(f'True Positive:{predict[target==1].sum()}, '
      f'True Negative:{(target==0).sum() - predict[target==0].sum()}, '
      f'False Positive:{predict[target==0].sum()}, '
      f'False Negative:{(target==1).sum() - predict[target==1].sum()}')

True Positive:1038221, True Negative:312866, False Positive:378552, False Negative:317291


In [14]:
accr = (tp+tn)/(tp+tn+fp+fn)
posaccr = tp/(tp+fp)
negaccr = tn/(tn+fn)
print(f'Accuracy:{accr}, '
      f'Positive Accuracy:{posaccr}, '
      f'Negative Accuracy:{negaccr}, '
      f'Weighted Accuracy:{0.5*(posaccr+negaccr)}')

Accuracy:0.6600553023308076, Positive Accuracy:0.7328068787307493, Negative Accuracy:0.4964889702090114, Weighted Accuracy:0.6146479244698804


# Bagging 9

In [15]:
with open('scrimmage4_link_dataset.pickle', 'rb') as file:
    link_dataset = pickle.load(file)

In [16]:
NUM_EPOCHS = 1
model_list = []
BATCH_SIZE = 1024

for model_idx in range(9):
    
    train_data = []
    for link_idx in np.random.choice(200,size=200,replace=True):
        train_data.append(link_dataset[link_idx])
    test_data = link_dataset[200:]
    
    train_x = torch.cat(tuple(link[0] for link in train_data),dim=0)
    train_y = torch.cat(tuple(link[1] for link in train_data),dim=0)
    test_x = torch.cat(tuple(link[0] for link in test_data),dim=0)
    test_y = torch.cat(tuple(link[1] for link in test_data),dim=0)

    model = MLP()
    model.cuda()

    loss_function = nn.CrossEntropyLoss(weight=torch.tensor([1.0,1.0])).cuda()
    optimizer = optim.Adam(model.parameters(), 0.001)

    train_dataloader = data.DataLoader(
        dataset=data.TensorDataset(train_x,train_y), 
        batch_size=BATCH_SIZE, shuffle=True, num_workers=16, pin_memory=True)

    test_dataloader = data.DataLoader(
        data.TensorDataset(test_x,test_y), 
        batch_size=BATCH_SIZE, shuffle=False, num_workers=16, pin_memory=True)

    model.train()
    for epoch_idx in range(NUM_EPOCHS):  # again, normally you would NOT do 300 epochs, it is toy data

        progress_training_epoch = tqdm(
            train_dataloader, 
            desc=f'Epoch {epoch_idx}/{NUM_EPOCHS}, Training',
            miniters=1, ncols=88, position=0,
            leave=True, total=len(train_dataloader), smoothing=.9)

        progress_validation_epoch = tqdm(
            test_dataloader, 
            desc=f'Epoch {epoch_idx}/{NUM_EPOCHS}, Validation',
            miniters=1, ncols=88, position=0, 
            leave=True, total=len(test_dataloader), smoothing=.9)

        train_loss = 0
        train_size = 0
        for idx, (input, target) in enumerate(progress_training_epoch):
            input = input.cuda()
            target = target.cuda()
            model.zero_grad()
            predict = model(input)
            loss = loss_function(predict, target)
            loss.backward()
            optimizer.step()
            train_loss += loss * target.size()[0]
            train_size += target.size()[0]

        test_loss = 0
        test_size = 0
        model.eval()
        with torch.no_grad():
            for idx, (batch_input, batch_target) in enumerate(progress_validation_epoch):
                batch_input = batch_input.cuda()
                batch_target = batch_target.cuda()
                batch_predict = model(batch_input)
                loss = loss_function(batch_predict, batch_target)
                test_loss += loss * batch_target.size()[0]
                test_size += batch_target.size()[0]

        print(f'train loss:{train_loss.item()/train_size: .5f}, '
              f'test loss:{test_loss.item()/test_size: .5f}')
        
    model_list.append(model) 

Epoch 0/1, Validation: 100%|████████████████████████| 1999/1999 [00:45<00:00, 44.06it/s]


train loss: 0.41673, test loss: 0.68330


Epoch 0/1, Validation: 100%|████████████████████████| 1999/1999 [00:35<00:00, 55.68it/s]


train loss: 0.42750, test loss: 0.68511


Epoch 0/1, Validation: 100%|████████████████████████| 1999/1999 [00:41<00:00, 48.24it/s]


...

train loss: 0.41263, test loss: 0.66237


Epoch 0/1, Validation: 100%|████████████████████████| 1999/1999 [00:42<00:00, 47.52it/s]


train loss: 0.42689, test loss: 0.65917


Epoch 0/1, Validation: 100%|████████████████████████| 1999/1999 [00:41<00:00, 47.69it/s]

train loss: 0.42820, test loss: 0.66678


# Test

In [17]:
predict_agg = []
target_agg = []

for model in model_list:
    
    test_dataloader = data.DataLoader(
        data.TensorDataset(test_x,test_y), 
        batch_size=BATCH_SIZE, shuffle=False, num_workers=16, pin_memory=True)

    progress_validation_epoch = tqdm(
        test_dataloader, 
        desc=f'Epoch {epoch_idx}/{NUM_EPOCHS}, Validation',
        miniters=1, ncols=88, position=0, 
        leave=True, total=len(test_dataloader), smoothing=.9)
    
    predict = []
    target = []
    model.eval()
    with torch.no_grad():
        for idx, (batch_input, batch_target) in enumerate(progress_validation_epoch):
            batch_input = batch_input.cuda()
            batch_target = batch_target.cuda()
            batch_predict = model(batch_input)
            predict.append(batch_predict.argmax(dim=1).cpu().numpy())
            target.append(batch_target.cpu().numpy())        
    predict = np.concatenate(predict, axis=0)
    target = np.concatenate(target, axis=0)
    
    predict_agg.append(predict)
    target_agg.append(target)

predict = (np.array(predict_agg).sum(axis=0) > 1.5)
target = target_agg[0]
tp = predict[target==1].sum()
tn = (target==0).sum() - predict[target==0].sum()
fp = predict[target==0].sum()
fn = (target==1).sum() - predict[target==1].sum()
print(f'True Positive:{tp}, '
      f'True Negative:{tn}, '
      f'False Positive:{fp}, '
      f'False Negative:{fn}')
#print(f'True Positive:{predict[target==1].sum()}, '
      #f'True Negative:{(target==0).sum() - predict[target==0].sum()}, '
      #f'False Positive:{predict[target==0].sum()}, '
      #f'False Negative:{(target==1).sum() - predict[target==1].sum()}')

Epoch 0/1, Validation: 100%|███████████████████████| 1999/1999 [00:07<00:00, 255.79it/s]


True Positive:1310658, True Negative:93481, False Positive:597937, False Negative:44854


In [18]:
accr = (tp+tn)/(tp+tn+fp+fn)
posaccr = tp/(tp+fp)
negaccr = tn/(tn+fn)
print(f'Accuracy:{accr}, '
      f'Positive Accuracy:{posaccr}, '
      f'Negative Accuracy:{negaccr}, '
      f'Weighted Accuracy:{0.5*(posaccr+negaccr)}')

Accuracy:0.685973140263712, Positive Accuracy:0.686713524870389, Negative Accuracy:0.6757581233961036, Weighted Accuracy:0.6812358241332463
